# Using Supabase as a vector store with LlamaIndex in Python

## Before you get started

You will need to have a supabase database with the vector store extension enabled and can follow the llamaindex [docs](https://docs.llamaindex.ai/en/stable/examples/vector_stores/SupabaseVectorIndexDemo/) to get there. Or you can just follow me.

![](assets/pgvector.jpg)

> Important: For your supabase project's password be sure to use a password that has no special characters or it will break the connection string you'll be using later.

# Prerequisites

## Install the packages

Note that I used Python 3.10.13 and this all worked for me. I tried it in 3.11 and 3.12 and didn't have the same luck.

In [1]:
!pip install llama-index
!pip install llama-index-vector-stores-supabase
!pip install python-dotenv
!pip install llama-index-llms-openai

### Get your `.env` file ready

We will use a `.env` to store two pieces of information. Note that it doesn't exist because it's in `.gitignore` so that you don't accidentally commit your secrets. The file should read with two lines like this:

```
OPENAI_API_KEY=sk-...
POSTGRES_CONNECTION_STRING=postgresql:...
```

The POSTGRES_CONNECTION_STRING is the connection string to your supabase database which has the format `postgresql://<user>:<password>@<host>:<port>/<db_name>` and you can get it by going first to your project settings and then to Database and then to Connection String.

![](assets/projsettings.jpg)

![](assets/postgres.jpg)


### Load the basic packages and key information


In [2]:
import logging
import sys
import os

# logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
# logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index.core import SimpleDirectoryReader, Document, StorageContext
from llama_index.core import VectorStoreIndex
from llama_index.vector_stores.supabase import SupabaseVectorStore
from dotenv import load_dotenv
import textwrap

load_dotenv()  # This loads the variables from .env
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
connection_string = os.getenv("POSTGRES_CONNECTION_STRING")

# Writing to the Supabase vector store

### Grab the documents in a directory and prep them for llama index

In [3]:
collection_name = "maeda_demo"
documents = SimpleDirectoryReader("./data/john_maeda/").load_data()
print(f"> {len(documents)} doc chunks written to vecs schema{collection_name} in supabase")

> 9 doc chunks written to vecs schemamaeda_demo in supabase


### Write documents to `collection_name` in supabase

> Warning: Be sure to delete the `collection_name` before adding documents to it, or else you'll keep accruing documents into that collection. You can either delete it manually like below, or use the code at the end of this notebook.

![](assets/deletecollection.jpg)

Let's go ahead and write the chunks to supabase.

In [4]:
vector_store = SupabaseVectorStore(
    postgres_connection_string=(
       connection_string
    ),
    collection_name=collection_name,
)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context
)

### You're done! Now you can read from the database any time you want.

# Reading from the Supabase vector store

### Let's plug llamaindex into the Supabase vector store

In [5]:
from llama_index.core import StorageContext, load_index_from_storage

vector_store = SupabaseVectorStore(
    postgres_connection_string=(
       connection_string
    ),
    collection_name=collection_name,
)

storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_vector_store(vector_store = vector_store, storage_context=storage_context)

## Query the index however you like

In [6]:
import warnings

# I added this to suppress a warning that stressed me out
warnings.filterwarnings("ignore", message="Query does not have a covering index for cosine_distance")

query_engine = index.as_query_engine()
query = """What is Maeda's george clooney inspiration? 
  What's the specific takeaway? 
  Don't make up any detail that isn't in the text.
"""
response = query_engine.query(query)
print(f"> {query}")
print(textwrap.fill(str(response), 100))

> What is Maeda's george clooney inspiration? 
  What's the specific takeaway? 
  Don't make up any detail that isn't in the text.

Maeda's George Clooney inspiration comes from an interview with George Clooney in Parade Magazine.
The specific takeaway from Clooney's interview is the importance of learning from failures, shifting
mindset to focus on solutions rather than seeking approval, and sometimes needing to fake confidence
while being open to failure.


It's a lot of fun to talk to oneself ... sorta.

In [7]:
query = """What did the author do growing up? 
  Say 'I don't know' if you don't know.
"""
response = query_engine.query(query)
print(f"> {query}")
print(textwrap.fill(str(response), 100))

> What did the author do growing up? 
  Say 'I don't know' if you don't know.

The author grew up in Seattle, Washington, and showed proficiency in both art and math.


## Retrieve the actual chunks related to the query

This is some utility code for setup.

In [13]:
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.postprocessor import LLMRerank
from llama_index.core import QueryBundle
from IPython.display import Markdown
def get_retrieved_nodes(query_str, vector_top_k=5, reranker_top_n=2, with_reranker=False):
    query_bundle = QueryBundle(query_str)
    # configure retriever
    retriever = VectorIndexRetriever(
        index=index,
        similarity_top_k=vector_top_k,
    )
    retrieved_nodes = retriever.retrieve(query_bundle)

    if with_reranker:
        # configure reranker
        reranker = LLMRerank(
            choice_batch_size=3,
            top_n=reranker_top_n,
        )
        retrieved_nodes = reranker.postprocess_nodes(retrieved_nodes, query_bundle)

    return retrieved_nodes

def pretty_print(results):
    markdown_str = "| Score | Text |\n|-------|------|\n"
    for result in results:
        # Replace line breaks with space for Markdown format
        text = result["Text"].replace("\n", " ")
        markdown_str += "| {} | {} |\n".format(result["Score"], text)
    display(Markdown(markdown_str))

def visualize_retrieved_nodes(nodes) -> None:
    result_dicts = []
    for node in nodes:
        result_dict = {"Score": node.score, "Text": node.node.get_text()}
        result_dicts.append(result_dict)

    pretty_print(result_dicts)

### Without Re-ranking it's not working that hard

In [9]:
my_new_query = "What did John Maeda think about the role of technology in art?"
nodes = get_retrieved_nodes(my_new_query)
visualize_retrieved_nodes(nodes)

| Score | Text |
|-------|------|
| 0.10057966456065359 | Key takeaways:  - Technology should be humanized and design in the digital age should be simple.  - A thriving 21st century economy must find its leaders in artists and designers. - Art and technology are interconnected and should be harnessed to push their boundaries.  - John Maeda comes from a manual labor background and grew up in a hardworking environment.  - The example set by his father taught him the value of craft and hard work.  - He learned about computers as they became more accessible and advanced.  - He believes creativity and art are worth something too, discovered at MIT.  - Computer programming was hand-crafted in the 80s.  - John Maeda was the only person who could both draw and program.  - He realized that there are others who can do this better, prompting him to strive to get better.  - John Maeda discovered art school accidentally.  - His father's rationale for him to finish his master's degree in semiconductor physics was so that he would be able to feed his family someday.  - While in art school, Maeda read a lot of Bauhaus and other old art history to learn about the relationship between art and science.  - Maeda discovered the idea that technology, science, and art can lead to a quantum leap.  - He found a teacher who encouraged him to do something younger with himself, which inspired him to pick up the computer and create art.  - Maeda's art was often seen as disgraceful by critics.  - He was asked by his teachers to become a professor and use his work to figure out if it was any good.  - Redesigning software to make it more applicable to designing art has been a long strand since the sixties.  - Maeda credits web design with making his art more mainstream.  - A. Michael Noll believed that making art was important, but policy makers have an influence over the whole space of the world.  - People are looking for a new kind of leader who can take risks, think creatively, work hard, and uphold the highest level of integrity, similar to the qualities and characteristics of an artist or designer.  - Artists often seek to participate in society on a grand scale and have the unique ability to imagine and create something out of something ordinary.  - At Google, employees are offered five-star cafeterias and encouraged to bring their dogs to work.  - Artists need to struggle to feel alive, and the ability to imagine is an extraordinary skill.  - Scientists and artists are similar in that both possess an integrity and commitment to discovery that doesn't scare them.  - The separation between art and science occurs because it's easier to compartmentalize and categorize.  - Funding for science projects is more refined than for arts projects, and, as such, artists can learn from scientists how to better raise funds for their explorations. - Funding for science is often dependent on a project's applicability and ability to be replicated.  - Art and science funding have drastically different budgets in comparison.  - Policy in the US is biased towards scientists when it comes to awarding  - The creative economy includes lawyers and all other people doing thinking work.  - It is driven by artists and designers creating businesses and experiences that people want to buy.  - Apple is successful because their well-designed product fits into people’s culture.  - Artists and designers are leaders in creating a fresh experiences that computers cannot produce.  - Social media is over-believed in and there is still more value in being together in person.  - It is important for people beyond art and design to understand what art and design really is.  - Art and design institutions should look outside the traditional definitions of their fields to how the world is looking for creative minds.  - Artists are sophisticated and creative thinkers who can change the economy.  - Designers make businesses and experiences.  - An event, such as a concert, causes people to move together “like lemmings” as they are all connected through their phones and the experience of being together. |
| 0.11176438172140901 | I said well I want to study the classics and be like you, teacher.  He looked at me like I was crazy and he said, âYou idiot, in Japanese; you horse's behind; that's pretty bad in Japanese.  He said you're young, so do something young with yourself.  When you get older, the classics will always be there, so don't worry about it so much.  So it was very freeing.  So I picked up the computer again; began to make stuff.  The stuff was unique because at the time people who could make art and computer program were a rarity and made stuff; people didn't know what it was.  People would stare at it and say that's not art; that's nothing; you stop doing that; it's a disgrace.  It was really good; it was good, like oh, you think that, well thank you. Kept on making stuff.  It began to become more mainstream.  My teachers at the time said to me that I should become a professor, because I should figure out if my work was any good or not.  Because at the time I was one of very few people who could do this.  I said well make more people that can do it; and see if you are really any good at it.  Make the people that will come and destroy you someday.  So at MIT I did that. I came in touch with all these people who could do things like this on the computer, much better than me.  And so I changed again.  Jo Reed:  You also redesigned software to in fact make it more applicable to designing art.   John Maeda: Yes, it's been a long strand since the sixties, computer art at Bell Laboratories; there's always been a movement to link computers with art, a long standing linkage. Until the web, there was no way to deliver it easily.  Jo Reed:  Not to us. It was happening in Hollywood in some ways.  John Maeda:  It was happening all over the place. Exactly, but it could never become mainstream, so I feel very lucky I was there when it hit mainstream.  I'll never forget when I was interviewed with Times one time, and I was asked, who's my idol.  And I said well this man, A. Michael Noll, how was at Bell Labs; ahead of his time; did some amazing stuff like virtual reality before computer screens existed; all this stuff.  And so I mentioned that.  And then a letter came in the mail from A. Michael Noll, and he said, I'm so glad someone out there remembers me still.  But the second thing he said, is he said that the reason why, see he stopped making art in the late sixties, went to work for Nixon I think; went into policy making.  He said he stopped making art because he realized that making art is important; but policy makers define the whole space of the world.  So he put all his work into policy making.  And I was like, huh, what's that?   Jo Reed:  Your latest book Redesigning Leadership really talks about what leaders can take from artists and designers, this seems like a perfect segue.  John Maeda:  I think that people are looking for new kinds of leaders, and they haven't been able to find them in business schools, or technology schools.  They're looking for folks that can take risks, can think around corners, that can work extremely hard and represent the highest form of integrity and if you lay those out, it sounds a lot like an artist; a designer.  So I've been looking at this role as President of RISD, as a kind of opportunity to just talk out loud about leadership and how being an artist and designer, helps to shape my experience, which I want to share with others who are grappling with what it means to be a leader in this very tumultuous times.  And how I draw upon my understanding of art and design to help me cope; help me succeed; and I'd like to share that.  Jo Reed:  You know there's often a stereotype I think of the artist; the lone individual, struggling away, and you know, certainly there is a lot of solitary work that happens with art, but as you point out, artists are also extraordinarily collaborative.  John Maeda:  Yes, yes.  Jo Reed:  They do work well with others.  John Maeda:  I know very few artists that want to live in a closet by themselves. |
| 0.11522990260336086 | And I found a book in the library, by a man named Paul Rand, a graphic designer and I opened up the book, and I looked at all the pictures, and the text, and I thought, wow I really suck at this.  People are really good at this, so how do I get better?  That's how it began.  Jo Reed:  And you ended up after MIT going to art school in Japan?  John Maeda:  Yes, a lot of accidents, happenstances more like â¦I got to MIT and in my first year there, the upper classmen were studying for this test, called the GRE and I was like, what's this test for, because I did the SAT thing, you know.  Well this is to go to graduate school.  And I called my dad and said, there's something called graduate school and so Dad said, go to graduate school, you know?  There was a Professor named Muriel Cooper who was the Art Director at MIT Press for a long time, and she saw my skills at work and she said, maybe you should be in art school instead.  And I thought, what is that? Never knew it actually existed.   I wanted to switch into something like this Rand stuff I saw; my father told me I'd have to finish doing what I'm doing.  His whole rationale was you'll have to feed your family someday, John.  So you know, that was his whole thing.  And I finished my Master's degree in semiconductor physics. He said well you can get a job now, you can support a family if you ever needed to; go do what you want to do now.  So I went to art school and changed my life.  Jo Reed:  Did you know immediately John that art and science had a lot to say to each other or was that something that you discovered?  John Maeda:  Not at all, I knew nothing about this.  Really at MIT, this media lab place was starting to take off, and so I hung out there. So, in art school I discovered- the school I went to was very Bauhaus-ish- it was Tsukuba, outside of Tokyo, and I just kept reading.  They had a large English language library.  So Bauhaus, old art history, going back way back.  I was like wow, there has always been this curiosity about art and how art advances the world and what's science-like collides with it, and something happens like the Futurists, the idea that the automobile could change the entire perspective of art.  I was like huh, technology, science and art have always led to some kind of quantum leap.  Photography, you know, the unholy art.  Later, we love photographs, so technology always puts a little kind of an accident in the oyster, a little bit of sand gets in there and you wait a while and artists, people create the art that we all suddenly can't live without.  Jo Reed:  During that time in art school were you working at all with computers, and if not, did you miss them?  John Maeda:  I think everything in my life has been like a very happy accident. Went to art school, and I was very happy âcause I was away from computers. Very happy.  Jo Reed: What were you doing?  John Maeda:  I was setting type, the old fashioned way; I was making sculptures out of aluminum, sitting there for a like aluminum is so hard; we think it's so soft, but it takes forever to mill aluminum by hand.  And then I had this teacher in Tsukuba, who pulled me aside and said, so what will you do with your life, John?  I said well I want to study the classics and be like you, teacher.  He looked at me like I was crazy and he said, âYou idiot, in Japanese; you horse's behind; that's pretty bad in Japanese.  He said you're young, so do something young with yourself.  When you get older, the classics will always be there, so don't worry about it so much.  So it was very freeing.  So I picked up the computer again; began to make stuff.  The stuff was unique because at the time people who could make art and computer program were a rarity and made stuff; people didn't know what it was.  People would stare at it and say that's not art; that's nothing; you stop doing that; it's a disgrace. |
| 0.11931509490203107 | It keeps moving and computers cannot create fresh experiences.  We, people; we are very sensate; artists feel the world. They craft/sculpt experiences every day.  They thrive to let you, the audience feel something.  You can't feel it if you feel the same thing every day.  So it's always a little bit different.  Jo Reed:  And in some ways, that goes to what you've said about what you've learned here as President of RISD, the buy-in doesn't happen to be a social media, it happens more on one.  John Maeda:  Yeah I was at this thing this week about best practices in the social media for fundraising. And so I was watching the expert talk about all the social media; this, that, whatever, you know, Twitter, Facebook, whatever, a conversion rates, etcetera, and for me, I couldn't help but feel like, I am so tired of social media; I mean it's like so like Tweeted out, you know I stopped my Facebook page, like last year and the reason why isn't because I don't believe in it. I believe we're over believing in it.  I was so excited to hear that you were going to be here in the room with me, not over the phone, so I could actually see you and feel you, understand you.  I thought wow; this is going to be much more exciting because I wanted to give you a better interview because of it; because you're here; because you bothered to come here, so thank you.  Jo Reed:  My pleasure.  John Maeda:  That's a big deal.  Jo Reed:  But it's also the truth; I'll always rather be there, because you do get a better interview; there's just no question about it.  John Maeda:  Yeah, I think that that's lost on the social media world.  The thing I found the most ironic, as the person spoke and I could hear other case studies about it.  There was something about how, I forget, it might be Lady Gaga, or who was it that raised a lot of money online; an online campaign that succeeded, versus most don't succeed, it turns outAnd they were all kind of like wondering like why?  Could it be the celebrity?  Could it be whatever?  It struck me how it's because in the case where money was raised, it was linked to an event.  You're all there in the concert space.  And so you're all there with your phone ready, and so it's actually a synchronous event; you're all there together, that's why you're all moving in like lemmings together.  It's like the famous story about the commercials for the Superbowl.  Like why is a commercial there so popular?  It's number one, a live event.  Number two, if it's live, they have to be there; they can TiVo it of course, but they're all there with their popcorn and barbeque ribs, whatever, and so the commercial gains more value because you're sitting there watching it knowing that someone else, millions of people are watching it at the same time, that's a rush.  We want to be together; even if we're apart, we want to be together.So social media is very asynchronous, and I think it's best when you're all at the same ti- which is not the whole point, the point is asynchronous.  I find that interesting.   Jo Reed:  So put on your predicting hat, and what would you…  John Maeda:  It's a very small hat.    Jo Reed:  Okay.  How about just a couple of ways that you think artists and designers can be given more value for their role in the creative economy.  John Maeda:  Oh, well two things:  The first thing is that the world beyond art and design, private equity people, new dot com-ers, policy makers, should know what art and design really is.  Artists are sophisticated people; they are creative thinkers.  Designers make businesses.  They change the economy.  Getting in touch with that fact is step number one.  The second challenge is inside art and design institutions; recognizing that there's a larger role to play.  I think that many people in institutions, like even mine, think of art and design a certain way. So inside institutions themselves, looking beyond the quoted version of art; the quoted version of design; what it means, to look more expansively to how the world is looking for these kinds of creative minds. |
| 0.12159254205003067 | John Maeda:  Well I think the grants sort of asking for exploration; and going back to the whole policy thing, I mean the thing that we discovered in Washington is that many of these grant-making procedures have certain requirements.  Maybe you need a PhD to get funding.  For most artists, a critical degree is an MFA; so there's  a kind of think stuck inside our policies for grant awarding that bias towards scientists. So that goes back to A. Michael Noll's point, that the policy holds the key to changing perceptions at a- not perceptions but even like, possibility.  So it isn't the fact that when your daughter comes up to you and says, you want to be an artist and your immediate reaction is, no you're not going to be an artist, you know?  That reaction isn't a reaction because artists are bad; it's because you're worried about their ability to feed themselves in the future.  But what if in the future, being an artist was as good as being a scientist, which isn't that bad, too.  That could change perceptions.  I think Europe had that angle and is losing it right now, because Europe saw the cultural value of art in relationship to cultural value of science, and now as if I just get tighter and we become more - a truly capitalistic society; you can't measure art's impact; you toss art out, in Europe.  America is learning how to realize that innovation doesn't come from the sky, it comes from innovative people that the art and design field has like in spades.  So why don't we use them?  Jo Reed:  You're a champion for this, for the role that artists and designers play in the 21st century creative economy, so I kind of want to approach this two ways, first, give me your definition of what the creative economy is; what does that mean to you?  John Maeda:  I like Richard Florida's definition of the creative class; the creative economy around that. Anyone doing thinking work is arguably creative.  And his definition that includes lawyers and all kinds of people, so I find that still valid. In particular though, in business creation; in creating new types of businesses, that's the creative economy on steroids.  Let me give you an example:  So there's a company Air BandB, it's what's called a couch surfing site; so if you have an extra room, you can rent out your room to anyone, an eBay for like your room.  That now has a billion dollar valuation.  Ashton Kutcher is now backing it as well; that's started by two of our graduates. You wouldn't expect that, but art and designers are now functioning as leaders in that economy that we know so clearly as the quote/unquote future.  You see it at that scale; you see it at even smaller scales; people making successfully their letter press work, and selling it on Etsy.com and making a living online.  That's also part of the economy.  Artists and designers make creatively, and people want to buy creatively.  Look at Apple.  Why is Apple so successful?  Not because they have the best product; they have the best-designed product.  It fits with your culture; they're making culture; they're artists in business.  Jo Reed:  Opening that iPhone, that box, it was beautiful.  John Maeda:  People love it.  Yeah.  And people keep trying to make something the exact same size, and the exact same like number of pixels, but it's more than that, it's the entire experience, and then experience is what artists and designers make, because experience always has to be fresh.  It keeps moving and computers cannot create fresh experiences.  We, people; we are very sensate; artists feel the world. They craft/sculpt experiences every day.  They thrive to let you, the audience feel something.  You can't feel it if you feel the same thing every day.  So it's always a little bit different.  Jo Reed:  And in some ways, that goes to what you've said about what you've learned here as President of RISD, the buy-in doesn't happen to be a social media, it happens more on one.  John Maeda:  Yeah I was at this thing this week about best practices in the social media for fundraising. |


### With Re-ranking it is a bit more accurate

In [14]:
new_nodes = get_retrieved_nodes(
    my_new_query,
    vector_top_k=3,
    with_reranker=True,
)
visualize_retrieved_nodes(new_nodes)

| Score | Text |
|-------|------|
| 10.0 | Key takeaways:  - Technology should be humanized and design in the digital age should be simple.  - A thriving 21st century economy must find its leaders in artists and designers. - Art and technology are interconnected and should be harnessed to push their boundaries.  - John Maeda comes from a manual labor background and grew up in a hardworking environment.  - The example set by his father taught him the value of craft and hard work.  - He learned about computers as they became more accessible and advanced.  - He believes creativity and art are worth something too, discovered at MIT.  - Computer programming was hand-crafted in the 80s.  - John Maeda was the only person who could both draw and program.  - He realized that there are others who can do this better, prompting him to strive to get better.  - John Maeda discovered art school accidentally.  - His father's rationale for him to finish his master's degree in semiconductor physics was so that he would be able to feed his family someday.  - While in art school, Maeda read a lot of Bauhaus and other old art history to learn about the relationship between art and science.  - Maeda discovered the idea that technology, science, and art can lead to a quantum leap.  - He found a teacher who encouraged him to do something younger with himself, which inspired him to pick up the computer and create art.  - Maeda's art was often seen as disgraceful by critics.  - He was asked by his teachers to become a professor and use his work to figure out if it was any good.  - Redesigning software to make it more applicable to designing art has been a long strand since the sixties.  - Maeda credits web design with making his art more mainstream.  - A. Michael Noll believed that making art was important, but policy makers have an influence over the whole space of the world.  - People are looking for a new kind of leader who can take risks, think creatively, work hard, and uphold the highest level of integrity, similar to the qualities and characteristics of an artist or designer.  - Artists often seek to participate in society on a grand scale and have the unique ability to imagine and create something out of something ordinary.  - At Google, employees are offered five-star cafeterias and encouraged to bring their dogs to work.  - Artists need to struggle to feel alive, and the ability to imagine is an extraordinary skill.  - Scientists and artists are similar in that both possess an integrity and commitment to discovery that doesn't scare them.  - The separation between art and science occurs because it's easier to compartmentalize and categorize.  - Funding for science projects is more refined than for arts projects, and, as such, artists can learn from scientists how to better raise funds for their explorations. - Funding for science is often dependent on a project's applicability and ability to be replicated.  - Art and science funding have drastically different budgets in comparison.  - Policy in the US is biased towards scientists when it comes to awarding  - The creative economy includes lawyers and all other people doing thinking work.  - It is driven by artists and designers creating businesses and experiences that people want to buy.  - Apple is successful because their well-designed product fits into people’s culture.  - Artists and designers are leaders in creating a fresh experiences that computers cannot produce.  - Social media is over-believed in and there is still more value in being together in person.  - It is important for people beyond art and design to understand what art and design really is.  - Art and design institutions should look outside the traditional definitions of their fields to how the world is looking for creative minds.  - Artists are sophisticated and creative thinkers who can change the economy.  - Designers make businesses and experiences.  - An event, such as a concert, causes people to move together “like lemmings” as they are all connected through their phones and the experience of being together. |
| 8.0 | I said well I want to study the classics and be like you, teacher.  He looked at me like I was crazy and he said, âYou idiot, in Japanese; you horse's behind; that's pretty bad in Japanese.  He said you're young, so do something young with yourself.  When you get older, the classics will always be there, so don't worry about it so much.  So it was very freeing.  So I picked up the computer again; began to make stuff.  The stuff was unique because at the time people who could make art and computer program were a rarity and made stuff; people didn't know what it was.  People would stare at it and say that's not art; that's nothing; you stop doing that; it's a disgrace.  It was really good; it was good, like oh, you think that, well thank you. Kept on making stuff.  It began to become more mainstream.  My teachers at the time said to me that I should become a professor, because I should figure out if my work was any good or not.  Because at the time I was one of very few people who could do this.  I said well make more people that can do it; and see if you are really any good at it.  Make the people that will come and destroy you someday.  So at MIT I did that. I came in touch with all these people who could do things like this on the computer, much better than me.  And so I changed again.  Jo Reed:  You also redesigned software to in fact make it more applicable to designing art.   John Maeda: Yes, it's been a long strand since the sixties, computer art at Bell Laboratories; there's always been a movement to link computers with art, a long standing linkage. Until the web, there was no way to deliver it easily.  Jo Reed:  Not to us. It was happening in Hollywood in some ways.  John Maeda:  It was happening all over the place. Exactly, but it could never become mainstream, so I feel very lucky I was there when it hit mainstream.  I'll never forget when I was interviewed with Times one time, and I was asked, who's my idol.  And I said well this man, A. Michael Noll, how was at Bell Labs; ahead of his time; did some amazing stuff like virtual reality before computer screens existed; all this stuff.  And so I mentioned that.  And then a letter came in the mail from A. Michael Noll, and he said, I'm so glad someone out there remembers me still.  But the second thing he said, is he said that the reason why, see he stopped making art in the late sixties, went to work for Nixon I think; went into policy making.  He said he stopped making art because he realized that making art is important; but policy makers define the whole space of the world.  So he put all his work into policy making.  And I was like, huh, what's that?   Jo Reed:  Your latest book Redesigning Leadership really talks about what leaders can take from artists and designers, this seems like a perfect segue.  John Maeda:  I think that people are looking for new kinds of leaders, and they haven't been able to find them in business schools, or technology schools.  They're looking for folks that can take risks, can think around corners, that can work extremely hard and represent the highest form of integrity and if you lay those out, it sounds a lot like an artist; a designer.  So I've been looking at this role as President of RISD, as a kind of opportunity to just talk out loud about leadership and how being an artist and designer, helps to shape my experience, which I want to share with others who are grappling with what it means to be a leader in this very tumultuous times.  And how I draw upon my understanding of art and design to help me cope; help me succeed; and I'd like to share that.  Jo Reed:  You know there's often a stereotype I think of the artist; the lone individual, struggling away, and you know, certainly there is a lot of solitary work that happens with art, but as you point out, artists are also extraordinarily collaborative.  John Maeda:  Yes, yes.  Jo Reed:  They do work well with others.  John Maeda:  I know very few artists that want to live in a closet by themselves. |


### Using a "tree summarizer" approach in conjunction with re-ranking

In [15]:
from llama_index.core.postprocessor import LLMRerank
query_engine = index.as_query_engine(
    similarity_top_k=10,
    node_postprocessors=[
        LLMRerank(
            choice_batch_size=5,
            top_n=2,
        )
    ],
    verbose=True,
    response_mode="tree_summarize",
)
response = query_engine.query(
    my_new_query,
)

print(textwrap.fill(str(response), 80))


John Maeda believed that technology should be humanized and that design in the
digital age should be simple. He also emphasized the interconnectedness of art
and technology, advocating for pushing the boundaries of both disciplines to
harness their potential.


### Using citations to get specific details

In [16]:
from llama_index.core.query_engine import CitationQueryEngine

query_text = "I would like to hear your thoughts about art and technology. "

response_dict = {}  # Initialize an empty dictionary for response

query_engine = CitationQueryEngine.from_args(
    index,
    similarity_top_k=3,
    citation_chunk_size = 1024
)
myq = query_text # + " If the facts are not available say, 'I do not know.'\n"
response = query_engine.query(myq)
print(textwrap.fill(f"Q: {myq}",80))
print("")
print(textwrap.fill(f"ai-johnmaeda response: {response}",80))
print(f"-- with {len(response.source_nodes)} citations.")
print("---")
# response_text = f"query: {myq} response: {response}\n"
# # print out all nodes text with the enumeration of node
# for i, node in enumerate(response.source_nodes):
#     response_text += f"**{i}**: {node.node.get_text()}\n"

# Create a dictionary to represent the response
response_dict = {
    "text": f"Query: {myq}",
    "citations": []
}

# Extract information from each node and add to the response dictionary
for i, node in enumerate(response.source_nodes):
    node_text = node.node.get_text()
    # extract metadata
    metadata = node.node.metadata
    print(f"---\n[{i}]\n")
    for key, value in metadata.items():
        print(f"{key}: {value}")
    print("")
    # Assuming node_text is a dictionary or can be converted to one
    # You may need to adjust this based on the actual structure of node.node.get_text()
    print(node_text)
    response_dict["citations"].append(node_text)


Q: I would like to hear your thoughts about art and technology.

ai-johnmaeda response: Art and technology are interconnected and should be
harnessed to push their boundaries [1]. John Maeda's experiences highlight the
intersection of art and technology, showcasing how his journey involved a
transition from working with computers to art school, where he discovered the
relationship between art and science [3]. This demonstrates the potential for
creativity and innovation when art and technology converge, leading to new
possibilities and advancements in both fields.
-- with 3 citations.
---
---
[0]

file_path: /Users/maeda/Documents/GitHub/MaedaSupaLlamaIndex/data/john_maeda/Designing at the Intersection of Art and Design.md
file_name: Designing at the Intersection of Art and Design.md
file_size: 32486
creation_date: 2024-10-12
last_modified_date: 2023-08-14

Source 1:
Key takeaways:

- Technology should be humanized and design in the digital age should be simple. 
- A thriving 21st cent

# Congratulations! You've made it through the notebook!

---

## Bonus: Supabase vec schema utilities that might help you

### List all tables in the vecs schema

In [ ]:
import psycopg2

# Connect to the database
conn = psycopg2.connect(connection_string)

try:
    with conn.cursor() as cur:
        cur.execute("""
            SELECT table_name 
            FROM information_schema.tables 
            WHERE table_schema = 'vecs'
        """)
        tables = cur.fetchall()
        
        print("Tables in 'vecs' schema:")
        for table in tables:
            print(f"- {table[0]}")

except Exception as e:
    print(f"An error occurred: {e}")
finally:
    conn.close()

### Delete a table in the vec schema

In [ ]:
import psycopg2

# Connect to the database
conn = psycopg2.connect(connection_string)
conn.autocommit = True

try:
    with conn.cursor() as cur:
        drop_table_sql = f'DROP TABLE IF EXISTS vecs."{collection_name}" CASCADE'
        cur.execute(drop_table_sql)
        print(f"Table 'vecs.{table_to_delete}' has been deleted successfully.")

except Exception as e:
    print(f"An error occurred: {e}")
finally:
    conn.close()